In [ ]:
import pycisTopic
import dill
from pycisTopic.diff_features import (
    impute_accessibility,
    normalize_scores,
    find_highly_variable_features,
    find_diff_features
)
import numpy as np

from pycisTopic.clust_vis import (
    find_clusters,
    run_umap,
    run_tsne,
    plot_metadata,
    plot_topic,
    cell_topic_heatmap
)

from pycisTopic.gene_annotation import (
    get_chrom_sizes_and_alias_mapping_from_ucsc
)

import tempfile
import os
import pyranges as pr
from pycisTopic.gene_activity import get_gene_activity

In [ ]:
lustre_path='/lustre/scratch127/cellgen/cellgeni/tickets/' + os.getcwd().split('/')[4]
lustre_path

In [ ]:
with open(lustre_path+"/work/pycistopic/results_pycistopic_combine/combined_cistopic_object.pkl", "rb") as f:
    cistopic = dill.load(f)
cistopic

In [ ]:
cistopic.cell_data.to_csv(lustre_path+"/work/pycistopic/results_pycistopic_combine/combined_cistopic_object_cell_data.csv")

In [ ]:
# it takes about 5 hours to run this model ub 16 cpu
os.environ['MALLET_MEMORY'] = '200G'
from pycisTopic.lda_models import run_cgs_models_mallet
# Configure path Mallet
mallet_path="/opt/Mallet/bin/mallet"
tmp_path = tempfile.mkdtemp()#'/tmp'
# Run models
models=run_cgs_models_mallet(
    cistopic,
    n_topics=[40],
    n_cpu=6,
    n_iter=500,
    random_state=555,
    alpha=50,
    alpha_by_topic=True,
    eta=0.1,
    eta_by_topic=False,
    tmp_path=tmp_path,
    save_path=tmp_path,
    mallet_path=mallet_path,
)

In [ ]:
from pycisTopic.lda_models import evaluate_models
model = evaluate_models(
    models,
    select_model = 40,
    return_model = True
)

In [ ]:
cistopic.add_LDA_model(model)

In [ ]:
run_umap(
    cistopic,
    target  = 'cell', scale=True)

In [ ]:
find_clusters(
    cistopic,
    target  = 'cell',
    k = 10,
    res = [0.6, 1.2, 3],
    prefix = 'pycisTopic_',
    scale = True,
    split_pattern = '-'
)

In [ ]:
import pickle
pickle.dump(
    cistopic,
    open(lustre_path+"/work/pycistopic/results_pycistopic_combine/combined_cistopic_object_add1.pkl", "wb")
)

In [ ]:
plot_metadata(
    cistopic_obj,
    reduction_name='UMAP',
    variables=['Seurat_cell_type', 'pycisTopic_leiden_10_0.6', 'pycisTopic_leiden_10_1.2', 'pycisTopic_leiden_10_3'],
    target='cell', num_columns=4,
    text_size=10,
    dot_size=5)

# Gene scores

In [ ]:
imputed_acc_obj = impute_accessibility(
    cistopic,
    selected_cells=None,
    selected_regions=None,
    scale_factor=10**6
)

In [ ]:
chromsizes = get_chrom_sizes_and_alias_mapping_from_ucsc(
    ucsc_assembly="hg38",
    chrom_sizes_and_alias_tsv_filename="hg38.chrom_sizes_and_alias.tsv",
)

In [ ]:
chromsizes = pd.read_table(os.path.join(out_dir, "qc", "hg38.chrom_sizes_and_alias.tsv"))
chromsizes.rename({"# ucsc": "Chromosome", "length": "End"}, axis = 1, inplace = True)
chromsizes["Start"] = 0
chromsizes = pr.PyRanges(chromsizes[["Chromosome", "Start", "End"]])
chromsizes

In [ ]:
pr_annotation = pd.read_table(
        os.path.join(out_dir, "qc", "../nf-atac/reference/hg38_pycistopic_tss.bed")
    ).rename(
        {"Name": "Gene", "# Chromosome": "Chromosome"}, axis = 1)
pr_annotation["Transcription_Start_Site"] = pr_annotation["Start"]
pr_annotation = pr.PyRanges(pr_annotation)
pr_annotation

In [ ]:
gene_act, weigths = get_gene_activity(
    imputed_acc_obj,
    pr_annotation,
    chromsizes,
    use_gene_boundaries=True, # Whether to use the whole search space or stop when encountering another gene
    upstream=[1000, 100000], # Search space upstream. The minimum means that even if there is a gene right next to it
                             # these bp will be taken (1kbp here)
    downstream=[1000,100000], # Search space downstream
    distance_weight=True, # Whether to add a distance weight (an exponential function, the weight will decrease with distance)
    decay_rate=1, # Exponent for the distance exponential funciton (the higher the faster will be the decrease)
    extend_gene_body_upstream=10000, # Number of bp upstream immune to the distance weight (their value will be maximum for
                          #this weight)
    extend_gene_body_downstream=500, # Number of bp downstream immune to the distance weight
    gene_size_weight=False, # Whether to add a weights based on the length of the gene
    gene_size_scale_factor='median', # Dividend to calculate the gene size weigth. Default is the median value of all genes
                          #in the genome
    remove_promoters=False, # Whether to remove promoters when computing gene activity scores
    average_scores=True, # Whether to divide by the total number of region assigned to a gene when calculating the gene
                          #activity score
    scale_factor=1, # Value to multiply for the final gene activity matrix
    extend_tss=[10,10], # Space to consider a promoter
    gini_weight = True, # Whether to add a gini index weigth. The more unique the region is, the higher this weight will be
    return_weights= True, # Whether to return the final weights
    project='Gene_activity') # Project name for the gene activity object

# Differentially Accessible Regions

In [10]:
normalized_imputed_acc_obj = normalize_scores(cistopic, scale_factor=10**4)

2025-05-19 14:18:28,023 cisTopic     INFO     Normalizing imputed data
2025-05-19 14:18:28,025 cisTopic     INFO     Done!


UnboundLocalError: cannot access local variable 'output' where it is not associated with a value

In [6]:
variable_regions = find_highly_variable_features(
    cistopic,
    min_disp = 0.05,
    min_mean = 0.0125,
    max_mean = 3,
    max_disp = np.inf,
    n_bins=20,
    n_top_features=None,
    plot=True
)

AttributeError: 'CistopicObject' object has no attribute 'mtx'

In [9]:
cistopic.mtx

AttributeError: 'CistopicObject' object has no attribute 'mtx'